# Imports

In [1]:
import bruce, numpy as np, matplotlib.pyplot as plt, os
from astropy.table import Table, Column
from astropy.stats import sigma_clip

/Users/sam/anaconda3/lib/python3.9/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/sam/.lightkurve/cache. Please move all the files in the legacy directory /Users/sam/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


# Load the tics

Data should have the form of (ascii or CSV, your choice). t_zero is in full BJD, width is in days, depth is in normalised flux. If you ue SPOCFIT, if you fit a single transit you will see 3 reported values on the bottom row which are these values for each event (t_zero, width, depth).

tic_id,	t_zero_1,	width_1,	depth_1,	t_zero_2,	width_2,	depth_2,

In [2]:
duos = Table.read('duos.csv')
duos['tic_id'] = duos['TIC ID']
print(duos)

  TIC ID  Linked TIC ID ...   tic_id 
--------- ------------- ... ---------
  7145074       7145074 ...   7145074
 20904104      20904104 ...  20904104
 22317640      22317640 ...  22317640
 32179255      32179255 ...  32179255
 37117064      37117064 ...  37117064
 42428568      42428568 ...  42428568
 49066806      49066806 ...  49066806
      ...           ... ...       ...
 39904176      39904176 ...  39904176
207501148     207501148 ... 207501148
116261487     116261487 ... 116261487
  5267885       5267885 ...   5267885
275267824     275267824 ... 275267824
286969201     286969201 ... 286969201
103095888     103095888 ... 103095888
Length = 97 rows


# Now the main worker function

This function does the following:
-  Load the latest TESS data
-  Flatten the lightcurve
-  Fit the events
-  Calcualte the aliases
-  Plot the permissable aliases
-  Create a report

In [5]:
for i in range(len(duos))[:]:
    if os.path.isfile(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i])) : continue

    # Create the output dir (we'll use this as cache for the data too)
    output_dir = os.getcwd() + '/{:}'.format(duos['tic_id'][i])
    os.system('mkdir -p {:}'.format(output_dir))

    # Now load the TESS data (SPOC, QLP)
    # We are not making our own here like TESSTPF, not yet anyway...
    # for data_type
    #   single_product -> all sectors together
    #   per_sector -> list of per-sector lightcurves
    #   northern_duos -> YEARS 2 and 4, then a list of other sectors 
    #   southern -> YEARS 1 and 3, then a list of other sectors (NOT IMPLEMENTED YET) 
    t, data,data_labels, base_dir =  bruce.ambiguous_period.download_tess_data(duos['tic_id'][i], 
                                                              max_sector=None, 
                                                                   use_ffi=True, 
                                                                   download_dir=None, 
                                                                   bin_length=0.5/24)
    
    # Now flatten the data
    for j, k in zip(data, data_labels):
        # Flatten the data by SG filter, we need an odd kernel legth based on cadence
        j.flatten_data_old(window_width=3, sigmaclip=3, dx_lim=0.1)


        for seg in bruce.data.find_nights_from_data(j.time, dx_lim=0.2):
            j.w = np.ones(j.time.shape[0])*np.median(j.flux)

        # Optinally save the data
        j.write_data(output_dir + '/' +'TESS_DATA_{:}.txt'.format(k))
        fig, ax = j.plot_segments(dx_lim=0.5)
        fig.savefig(output_dir + '/' + 'TESS_DATA_{:}.png'.format(k))
        plt.close(fig)

    # # Now re-order_datasets based on epochs given
    # We will unpack now, (data with transits, and data without)
    # We may need to change this for it to work properly (Sam is working on it)
    # Its worth noting we can incorparate ground based data here too
    # data_from_ground = bruce.ambiguous_period.mono_event.photometry_time_series(time, flux, flux_err, w = norm_model)
    # Then this can go in data_other_sectors
    data, data_labels = bruce.ambiguous_period.group_data_by_epochs(data, data_labels, duos['t_zero_1'][i], duos['t_zero_2'][i])
    data, data_other_sectors = data[0], data[1:]



    ############################
    # FIT EVENT 1
    ############################
    # Mask data and create the mono_event object
    nmask = 3
    mask1 = (data.time > (duos['t_zero_1'][i] - nmask*duos['width_1'][i])) &  (data.time < (duos['t_zero_1'][i] + nmask*duos['width_1'][i]))
    data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], data.flux_err[mask1], w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    m1 = bruce.ambiguous_period.mono_event(duos['t_zero_1'][i], duos['width_1'][i], duos['depth_1'][i], data_event_1, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_1 = m1.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)


    ############################
    # FIT EVENT 2
    ############################
    # Mask data and create the mono_event object
    mask2 = (data.time > (duos['t_zero_2'][i] - nmask*duos['width_2'][i])) &  (data.time < (duos['t_zero_2'][i] + nmask*duos['width_2'][i]))
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], data.flux_err[mask2], w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    m2 = bruce.ambiguous_period.mono_event(duos['t_zero_2'][i], duos['width_2'][i], duos['depth_2'][i], data_event_2, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_2 = m2.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)

    # We are going to make a nice plot of the two events with their models
    fig, ax = plt.subplots(1,2, gridspec_kw={'hspace' : 0, 'wspace' : 0}, figsize = (6.4, 3.8))
    ax[0].errorbar(return_data_1[0], return_data_1[1], yerr=return_data_1[2], fmt='k.', alpha = 0.1)
    ax[0].plot(return_data_1[3], return_data_1[4], c='orange')
    ax[1].errorbar(return_data_2[0], return_data_2[1], yerr=return_data_2[2], fmt='k.', alpha = 0.1)
    ax[1].plot(return_data_2[3], return_data_2[4], c='orange')
    ax[1].set(yticks=[])
    ylim1 = ax[0].get_ylim()
    ylim2 = ax[1].get_ylim()
    ylim = [min(ylim1[0],ylim2[0]), max(ylim1[1], ylim2[1])]
    ax[0].set_ylim(ylim)
    ax[1].set_ylim(ylim)
    fig.supxlabel('Time from Transit [d]', fontsize=18, x=0.55, y = -0.005)
    fig.supylabel('Flux', fontsize=18)
    fig.suptitle(m2.name, y=0.95, x=0.55, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3', alpha=1.0), ha='center', fontsize=18)
    plt.subplots_adjust(right=0.99, top=0.99, bottom=0.13)
    fig.savefig(output_dir + '/' + 'TIC-{:}_BOTH_EVENTS.png'.format(duos['tic_id'][i]))
    plt.close(fig)

    ########################################################
    # CREATE THE AMBIGUOUS PERIOD OBJECT
    ########################################################
    p = bruce.ambiguous_period.ambiguous_period(data, events=[m1,m2], name='TIC-{:}'.format(duos['tic_id'][i]),
                        median_bin_size = 2,convolve_bin_size = 2)

    # Now mask and filter 
    p.mask_and_filter_events()

    # Calculate aliases
    # Do not use nsolutions_events here (that is superceeded later)
    nsolutions_events = p.calcualte_aliases(dx_lim=0.03, min_period=15)

    # Now calcualte whether we saw a transit by comparing the model to a flat line
    delta_L_data = p.calcualte_data_delta_L(data)
    #delta_L_data = delta_L_data + 100 # THIS FUDGE IS OFTEN NEEDED
    for j in range(delta_L_data.shape[0]):
        print(j, delta_L_data[j])
    delta_L_data_from_other_sectors_or_others = [p.calcualte_data_delta_L(j) for j in data_other_sectors]
    p.delta_L = np.array([delta_L_data, *delta_L_data_from_other_sectors_or_others])


    ########################################################
    # Plot the aliases
    ########################################################
    fig, ax, alias_colours  = p.plot_aliases(phot_data=data_other_sectors, phot_data_labels=data_labels)
    fig.savefig(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i]), dpi=300)
    plt.close(fig)
    
    ########################################################
    # Now report the aliases
    ########################################################
    aliases = (p.aliases)[alias_colours[:,-1]==alias_colours.max()]
    periods = (p.max_period/p.aliases)[alias_colours[:,-1]==alias_colours.max()]
    alaises = Table()
    alaises.add_column(Column(m1.de_get_epoch()*np.ones(len(aliases)), name='t_zero_1'))
    alaises.add_column(Column(m1.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_1'))
    alaises.add_column(Column(m1.de_get_k()*np.ones(len(aliases)), name='k_1'))
    alaises.add_column(Column(m1.de_get_b()*np.ones(len(aliases)), name='b_1'))
    alaises.add_column(Column(m1.de_transit_width()*np.ones(len(aliases)), name='width_1'))
    alaises.add_column(Column(m2.de_get_epoch()*np.ones(len(aliases)), name='t_zero_2'))
    alaises.add_column(Column(m2.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_2'))
    alaises.add_column(Column(m2.de_get_k()*np.ones(len(aliases)), name='k_2'))
    alaises.add_column(Column(m2.de_get_b()*np.ones(len(aliases)), name='b_2'))
    alaises.add_column(Column(m2.de_transit_width()*np.ones(len(aliases)), name='width_2'))
    alaises.add_column(Column(aliases, name='alias'))
    alaises.add_column(Column(periods, name='period'))
    alaises.write(output_dir + '/' + 'TIC-{:}_ALIASES.fits'.format(duos['tic_id'][i]), overwrite=True)

        
        


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpeaitbmln

Querying TIC 7145074 from MAST...



✅ Download summary:
Sector ...
------ ...
     2 ...
    29 ...
    69 ...
    96 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpeaitbmln
This directory will be deleted when the program exits.
W :  144
W :  144


W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
Initial Chi-Sqaured : 3226.87 [red 52.05]
Fitted parameters for TIC-7145074:
t_zero : 2459111.5180272562
radius_1 : 0.05051254275215275
k : 0.24981450464382335
b : 1.1523826329793194
Final Chi-Sqaured : 1433.49 [red 23.12]
Initial Chi-Sqaured : 357.17 [red 7.29]
Fitted parameters for TIC-7145074:
t_zero : 2460185.154238245
radius_1 : 0.04191172128157765
k : 0.09168760898549022
b : 0.9324691175826946
Final Chi-Sqaured : 73.99 [red 1.51]
1 72 1 71.57574739924942 1073.6362109887414 15 15.121636774489316
0 0.0
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 -0.6554050686880086
46 -3338.8769330245536
47 -7119.692445093252
48 -1097.0794412430955
49 -1270.431485712707
50 -

100%|███████████████████████████████████████████| 71/71 [00:06<00:00, 10.84it/s]



📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpi5w70wy_

Querying TIC 20904104 from MAST...


...

100%|███████████████████████████████████████████| 48/48 [00:05<00:00,  8.46it/s]



📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp43sx7tgo

Querying TIC 103095888 from MAST...



✅ Download summary:
Sector ...
------ ...
    19 ...
    25 ...
    26 ...
    52 ...
    53 ...
    59 ...
    73 ...
    79 ...
    86 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp43sx7tgo
This directory will be deleted when the program exits.
W :  143
W :  143


W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  28
W :  10
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  12
W :  144
W :  144
W :  13
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  70
W :  36
W :  62
W :  144
W :  20
W :  28
W :  144
Initial Chi-Sqaured : 138.11 [red 2.82]
Fitted parameters for TIC-103095888:
t_zero : 2459015.8288978385
radius_1 : 0.029782405364968664
k : 0.03999689186511039
b : 0.6412946012587569
Final Chi-Sqaured : 103.09 [red 2.10]
Initial Chi-Sqaured : 254.83 [red 7.08]
Fitted parameters for TIC-103095888:
t_zero : 2459746.006286069
radius_1 : 0.03424446315761917
k : 0.03874798837209946
b : 0.6921170379985089
Final Chi-Sqaured : 185.46 [red 5.15]
1 49 1 48.67849254868925 730.1773882303387 15 15.21202892146539
0 0.0
1 0.0
2 0.0
3 -0.007510053893662133
4 0.0
5 0.0
6 0.0
7 -0.007510053893669134
8 0.0
9 0.0
10 2.2839825330088477
11 -0.007510053893678703
12 0.0
13 0.0
14 1.5620613204046743
15 -0.007510053893671348
16 0.0
17 0.0
18 1.6

100%|███████████████████████████████████████████| 48/48 [00:12<00:00,  3.91it/s]


# Now lok for solved systems

In [13]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p solved_systems; rm solved_systems/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==1:
        os.system('cp {:}/TIC-{:}_ALIASES.png solved_systems'.format(tic_id,tic_id))
        
        print('{:}, {:}, {:}'.format(tic_id,  t['t_zero_1'][0], t['period'][0]))
        

49066806, 2459290.1263804263, 27.65127434802276
289840544, 2458331.289873723, 25.930949741075263
207501148, 2459007.727905593, 166.03730747115333
145006304, 2458579.5012627575, 15.07637889701303
142278054, 2458595.025455752, 26.493703934191554
330799539, 2458918.306298589, 23.968420412857085
209006017, 2459376.0885753254, 21.369980819701382
437293313, 2458578.44183257, 24.98815684582417
37117064, 2458483.095874101, 22.30350940712643
287204963, 2458402.9369503283, 38.389474153837284
276376289, 2458716.9436017587, 30.116887344526024
170692379, 2458643.309522811, 31.282666421752978
257024338, 2458922.4816428316, 81.43381791616169
229476204, 2458743.521617817, 648.0597211997956
270771827, 2458523.000238604, 20.439924176762695


array([734.80306314, 367.40153157, 244.93435438, 183.70076578,
       146.96061263, 122.46717719, 104.97186616,  91.85038289,
        81.64478479,  73.48030631,  66.80027847,  61.23358859,
        56.52331255,  52.48593308,  48.98687088,  45.92519145,
        43.2237096 ,  40.8223924 ,  38.67384543,  36.74015316,
        34.99062205,  33.40013923,  31.94795927,  30.6167943 ,
        29.39212253,  28.26165627,  27.21492826,  26.24296654,
        25.33803666,  24.49343544,  23.70332462,  22.96259572,
        22.26675949,  21.6118548 ,  20.99437323,  20.4111962 ,
        19.85954225,  19.33692271,  18.84110418,  18.37007658,
        17.92202593,  17.49531103,  17.08844333,  16.70006962,
        16.32895696,  15.97397963,  15.63410773,  15.30839715])

In [25]:
m1.de_transit_width()

0.12027835950740756